In [1]:
import trimesh
import potpourri3d as pp3d
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image

In [2]:
def get_scalar_heat_geodesic(mesh, vertex_indices):
    heat_solver = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
    return heat_solver.compute_distance_multisource(vertex_indices)

In [3]:
def get_logmap(mesh, source_vertex):
    heat_solver = pp3d.MeshVectorHeatSolver(mesh.vertices, mesh.faces)
    return heat_solver.compute_log_map(source_vertex)

In [7]:
def get_colors_from_image(image_path, coordinates):
    image = Image.open(image_path)
    image = image.convert("RGB")  # Ensure the image is in RGB format
    img_array = np.array(image)  # Convert the image to a NumPy array for pixel access

    height, width, _ = img_array.shape

    def get_bilinear_color(x, y):
        # Calculate the integer parts and fractional parts
        x0, y0 = int(x), int(y)
        dx, dy = x - x0, y - y0

        # Handle boundary cases
        if x < 0 or y < 0 or x > width - 1 or y > height - 1 :
            return np.array([200,200,200])
        
        x1 = min(x0 + 1, width - 1)
        y1 = min(y0 + 1, height - 1)

        # Get the pixel values at the four corners
        top_left = img_array[y0, x0]
        top_right = img_array[y0, x1]
        bottom_left = img_array[y1, x0]
        bottom_right = img_array[y1, x1]

        # Perform bilinear interpolation
        top = (1 - dx) * top_left + dx * top_right
        bottom = (1 - dx) * bottom_left + dx * bottom_right
        interpolated_color = (1 - dy) * top + dy * bottom

        return np.array(interpolated_color.astype(int))  # Convert to integer tuple (R, G, B)

    # Scale coordinates to image dimensions and get colors
    colors = np.array([get_bilinear_color(x * (width - 1), y * (height - 1)) for x, y in coordinates])
    return colors

In [8]:
def transform_coordinates(coords, sx, sy, tx, ty, theta):
    # Create the transformation matrix
    transformation_matrix = np.array([
        [1/sx * np.cos(theta), -1/sx * np.sin(theta), tx],
        [1/sy * np.sin(theta),  1/sy * np.cos(theta), ty],
        [0,                  0,                  1]
    ])
    
    # Convert the coordinates to homogeneous form (add a third dimension of 1s)
    homogeneous_coords = np.hstack([coords, np.ones((coords.shape[0], 1))])
    
    # Apply the transformation
    transformed_coords = homogeneous_coords @ transformation_matrix.T
    
    # Return the first two columns (x', y')
    return transformed_coords[:, :2]

In [9]:
def show_mesh(mesh, log_map, image_path):
    plotter = pv.Plotter()
    faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
    pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

    sources = np.array([11000])
    plotter.add_points(mesh.vertices[sources].reshape(sources.shape[0], 3), color="red", render_points_as_spheres=True, point_size=10)

    log_map = transform_coordinates(log_map, 2.5, 2.5, 0, 0, -2.2)
    colors = get_colors_from_image(image_path, log_map)
    plotter.add_mesh(pv_mesh, scalars=colors, rgb=True)
    plotter.show(title="Geodesic Distance Visualization")
    plotter.close()

In [10]:
mesh_name = 'human.obj'
mesh = trimesh.load(mesh_name)
log_map = get_logmap(mesh, 11000)
show_mesh(mesh, log_map, 'spiderman-logo.jpg')

Widget(value='<iframe src="http://localhost:58371/index.html?ui=P_0x24d3686fe50_1&reconnect=auto" class="pyvis…